<a href="https://colab.research.google.com/github/abyanrizz/Midterm_MLDL/blob/main/01.%20Trasaction/%20midterm1_v2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gdown
!gdown --folder 1tHVHE6kVxnc3n8HL4fw9-KEHDo8QYMQc?hl=id

Retrieving folder contents
Processing file 1VW9FTnesgZGlm8roxHpFK4S42AUlLS2P train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1VW9FTnesgZGlm8roxHpFK4S42AUlLS2P
From (redirected): https://drive.google.com/uc?id=1VW9FTnesgZGlm8roxHpFK4S42AUlLS2P&confirm=t&uuid=c4b8f2cc-16a2-4be9-b20e-11fb57d46cae
To: /content/Train_Transaction/train_transaction.csv
100% 683M/683M [00:09<00:00, 73.4MB/s]
Download completed


In [ ]:
import pandas as pd
import polars as pl
BASE_PATH = "Train_Transaction"
train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
print(train_transaction.shape) #590540 row dan 393 feature + 1 target

(590540, 394)


In [ ]:
train_transaction.head()

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd

df = train_transaction.to_pandas()
df.shape


(590540, 394)

In [ ]:
# Separate numeric and non-numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns
object_cols = df.select_dtypes(include=['object']).columns

# Fill missing values in numeric columns with their median
for col in numeric_cols:
    if df[col].isnull().any(): # Only attempt if there are NaNs
        df[col] = df[col].fillna(df[col].median())

# Fill missing values in object/categorical columns with their mode
for col in object_cols:
    if df[col].isnull().any(): # Only attempt if there are NaNs
        # Check if mode is empty (e.g., all NaNs or no clear mode)
        if not df[col].mode().empty:
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna('Missing') # Fallback for columns with no mode or all NaNs

In [ ]:
X = df.drop(columns=["isFraud"])
y = df["isFraud"]


In [ ]:
import polars as pl

exclude_cols = ["isFraud"]

# Get names of all numeric columns in train_transaction
numeric_polars_cols = train_transaction.select(pl.selectors.numeric()).columns

# Filter out the 'isFraud' column if it's numeric
numeric_cols_for_agg = [col for col in numeric_polars_cols if col not in exclude_cols]

mean_exprs = [pl.col(col).mean().alias(f"{col}_mean") for col in numeric_cols_for_agg]

std_exprs = [pl.col(col).std().alias(f"{col}_std") for col in numeric_cols_for_agg]

stats = train_transaction.select(mean_exprs + std_exprs)

In [ ]:
stats = stats.to_dict(as_series=False)


In [ ]:
scaled = train_transaction.with_columns([
    ((pl.col(col) - stats[f"{col}_mean"][0]) / stats[f"{col}_std"][0]).alias(col)
    for col in numeric_cols_for_agg # Iterate only over numeric columns
])

In [ ]:
df = scaled.to_pandas()


In [ ]:
X = df.drop(columns=['isFraud'])
y = df['isFraud']
